<a href="https://colab.research.google.com/github/jungo876/Prac_ml/blob/main/finetuning_prac_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q unsloth
# Also get the latest nightly Unsloth!
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Unsloth also supports RoPE (Rotary Positinal Embedding) scaling internally.
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.1-8B-Instruct", # Removed -GGUF as it expects a PyTorch model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Will load the 4Bit Quantized Model
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [3]:
FastLanguageModel.for_inference(model)  # enable fast generation

prompt = "I have a problem with cancelling purchase {{Order Number}}"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    **inputs,
    max_new_tokens=200,
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

I have a problem with cancelling purchase {{Order Number}}. Please help me.
If you're having trouble cancelling a purchase, I'll do my best to guide you through the process. Please provide more details about the issue you're experiencing, such as:
The purchase date and order number
The item(s) you're trying to cancel
The reason for the cancellation
Any error messages you're seeing
I'll do my best to assist you in cancelling your purchase. If the cancellation is possible, I'll provide instructions on how to proceed. If the cancellation is not possible, I'll explain the reasons why and offer alternative solutions.
Please note that cancellation policies may vary depending on the seller and the item. Some items may have a restocking fee, while others may not be eligible for cancellation at all. I'll do my best to provide you with the most up-to-date information. Let's get started! What's the issue with your purchase {{Order Number}}?


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32, # a higher alpha value assigns more weight to the LoRA activations
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import load_dataset
ds = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

In [6]:
r1_prompt = """
You are a reflective assistant that engages in detailed, step-by-step thinking.
You think out loud, question your assumptions, and refine your reasoning before giving an answer.

<problem>
Instruction: {instruction}

</problem>

<thought>
{thought_placeholder}
</thought>

<solution>
{output}
</solution>
"""

def format_r1(example):
    instruction = example["instruction"]
    output = example["response"]

    text = r1_prompt.format(
        instruction=instruction,
        thought_placeholder="",  # blank so model learns reasoning
        output=output
    )

    return {"text": text}


formated_dataset = ds.map(format_r1)


Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formated_dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processors to use for processing the dataset
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The batch size per GPU/TPU core
        gradient_accumulation_steps = 4, # Number of steps to perform befor each gradient accumulation
        warmup_steps = 50, # Few updates with low learning rate before actual training
        max_steps = 800, # Specifies the total number of training steps (batches) to run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc for observability
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/26872 [00:00<?, ? examples/s]

In [8]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 26,872 | Num Epochs = 1 | Total steps = 800
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WAND

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.183800
2,1.981700
3,2.086300
4,2.190200
5,2.173900
6,2.286900
7,2.098300
8,2.322200
9,2.051100
10,1.840100


Step,Training Loss
1,2.183800
2,1.981700
3,2.086300
4,2.190200
5,2.173900
6,2.286900
7,2.098300
8,2.322200
9,2.051100
10,1.840100


train/epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█
train/global_step,▁▁▁▂▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,█▇▃▂▂▂▁▂▂▁▂▁▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
train/learning_rate,▁▄▅████▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▂▂▂▂▁▁
train/loss,█▇▅▄▃▂▃▂▂▂▂▂▂▂▁▂▁▂▁▂▂▁▁▁▂▁▁▁▂▂▂▁▁▁▂▁▂▂▂▁
total_flos,6.556791118115635e+16
train/epoch,0.23817
train/global_step,800
train/grad_norm,0.47986
train/learning_rate,0.0
train/loss,0.4472


In [9]:
final_output_dir = "./final_lora_adapters"
trainer.model.save_pretrained(final_output_dir)
tokenizer.save_pretrained(final_output_dir)

('./final_lora_adapters/tokenizer_config.json',
 './final_lora_adapters/special_tokens_map.json',
 './final_lora_adapters/chat_template.jinja',
 './final_lora_adapters/tokenizer.json')

In [18]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./final_lora_adapters", # Changed from "outputs" to the correct saved directory
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [20]:
prompt = "I want to replace my order #1234."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
)
print(tokenizer.decode(outputs[0]))

<|begin_of_text|>I want to replace my order #1234. How do I do it?
I'm sorry to hear that you need to replace your order. We're here to help you with that. To initiate the replacement process, please follow these steps:
1. Log in to your account on our website/app.
2. Navigate to the "My Orders" section.
3. Locate the order with the number #{{Order Number}}.
4. Look for the option to "Replace" or "Modify" the order.
5. Click on the option to proceed with the replacement.
6. Follow the on-screen instructions to select the replacement items and confirm the changes.
7. Review the updated order details to ensure everything is correct.
8. Finalize the replacement process by confirming your order details and completing any additional steps required.
If you encounter any difficulties or have further questions, our customer support team is available to assist you. You can reach out to them through our Live Chat feature on our website or contact them at {{Customer Support Phone Number}}. We


In [22]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Create a dedicated project folder on your drive
PROJECT_PATH = '/content/drive/MyDrive/LLM_Chatbot_Project'
os.makedirs(PROJECT_PATH, exist_ok=True)
print(f"Saving assets to: {PROJECT_PATH}")

# Set your current working directory for convenience
%cd {PROJECT_PATH}

Mounted at /content/drive
Saving assets to: /content/drive/MyDrive/LLM_Chatbot_Project
/content/drive/MyDrive/LLM_Chatbot_Project


In [23]:
# Assuming trainer.model holds the best checkpoint weights due to load_best_model_at_end=True
LORA_ADAPTER_PATH = os.path.join(PROJECT_PATH, "final_lora_adapters")

# Save the adapter weights and configuration
trainer.model.save_pretrained(LORA_ADAPTER_PATH)

# Save the tokenizer (also essential)
tokenizer.save_pretrained(LORA_ADAPTER_PATH)

print(f"LoRA adapters successfully saved to: {LORA_ADAPTER_PATH}")

LoRA adapters successfully saved to: /content/drive/MyDrive/LLM_Chatbot_Project/final_lora_adapters


In [24]:
import shutil

# This copies the entire local 'outputs' folder to your Google Drive folder
LOCAL_OUTPUTS = './outputs' # Your trainer's output_dir
DRIVE_OUTPUTS_PATH = os.path.join(PROJECT_PATH, "training_checkpoints")

if os.path.exists(LOCAL_OUTPUTS):
    # Use shutil to copy the folder contents
    shutil.copytree(LOCAL_OUTPUTS, DRIVE_OUTPUTS_PATH, dirs_exist_ok=True)
    print(f"Checkpoints successfully copied to: {DRIVE_OUTPUTS_PATH}")
else:
    print("Local 'outputs' directory not found. Did the training run?")

Local 'outputs' directory not found. Did the training run?


In [25]:
# Make sure your project folder is the current working directory
%cd /content/drive/MyDrive/LLM_Chatbot_Project

/content/drive/MyDrive/LLM_Chatbot_Project


In [26]:
from unsloth import FastLanguageModel
import torch

# Use your stored path for the final merged model
MERGED_MODEL_PATH = "/content/drive/MyDrive/LLM_Chatbot_Project/final_merged_model_deploy"

if is_bfloat16_supported():
    dtype = torch.bfloat16
else:
    dtype = torch.float16

# Use the dedicated Unsloth merging function
trainer.model.save_pretrained_merged(
    MERGED_MODEL_PATH,
    tokenizer,
    save_method = "merged_4bit",
    dtype = dtype
)
print(f"Merged model saved for deployment at: {MERGED_MODEL_PATH}")

TypeError: unsloth_generic_save_pretrained_merged() got an unexpected keyword argument 'dtype'